In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# Get miRNA from TargetScan
df  = pd.read_csv('D:/ML4RG_file/genomic_sequence_plus_features_hl_all_tissues.csv', index_col = 0)
rownames = df.index
mirRNA_list = []
for r in rownames[0:10]:
    mirRNA = []
    url = 'http://www.targetscan.org/cgi-bin/targetscan/vert_72/view_gene.cgi?rs=' + r + '&taxid=9606&showcnc=0&shownc=0&shownc_nc=&showncf1=&showncf2=&subset=1'
    if requests.post(url).status_code == 200:
        html = requests.get(url)
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, "html.parser")
        table = soup.find_all('table', id='alt_tstable')
        if len(table)>0:
            line = table[0].find_all('tr')
            for l in line[1:]:
                mirRNA.append(l.text.split()[5][3:])
    print(mirRNA)
    mirRNA_list.append(mirRNA)

df_miRNA = pd.DataFrame()
df_miRNA['id'] = rownames
df_miRNA['miRNA'] = mirRNA_list
df_miRNA.to_csv('C:/Users/yuyu/Desktop/result_miRNA.csv')

In [47]:
def strTolist(s):
    if s != '[]':
        new_list = []
        element_list = s[1:-1].split(', ')
        for e in element_list:
            new_list.append(e[1:-1])
    else:
        new_list = []
    return new_list

# read a new df
df_RNA = pd.read_csv('C:/Users/yuyu/Desktop/result_miRNA.csv')
miRNA = df_RNA['miRNA']

# get all miRNA's name for each gene in a list 
all_miRNA = []
for mi in miRNA:
    miRNA_list = strTolist(mi)
    if len(miRNA_list)>0:
        for e in miRNA_list:
            if not e.split('.')[0] in all_miRNA:
                all_miRNA.append(e.split('.')[0])
                                     
# crreat a dictionary for {miRNA'name: miRNA family}, get family info from miRBase
miRNA_family_dict = {}

for mir,i in zip(all_miRNA,range(0,len(all_miRNA))):
    family = ''
    url = 'http://www.mirbase.org/cgi-bin/mirna_entry.pl?acc=' + mir
    html = requests.get(url)
    html.encoding = 'utf-8'
    soup = BeautifulSoup(html.text, "html.parser")
    table = soup.find_all('table', class_='data')
    if len(table)>0:
        left = [i.text for i in table[0].find_all('td', class_='left')]
        if 'Gene family' in left:
            family = table[0].find_all('td', class_='right')[left.index('Gene family')].text.split('; ')[1]
        else:
            family =  mir.split('-')[1].lower() + '-' + mir.split('-')[2]
    else:
        mir_1 = mir.split('-')[0] + '-' + mir.split('-')[1] + '-' + mir.split('-')[2]
        url = 'http://www.mirbase.org/cgi-bin/mirna_entry.pl?acc=' + mir_1
        html = requests.get(url)
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, "html.parser")
        table = soup.find_all('table', class_='data')
        if len(table)>0:
            left = [i.text for i in table[0].find_all('td', class_='left')]
            if 'Gene family' in left:
                family = table[0].find_all('td', class_='right')[left.index('Gene family')].text.split('; ')[1]
            else:
                family =  mir.split('-')[1].lower() + '-' + mir.split('-')[2]

    miRNA_family_dict.update({mir:family})

# get unique miRNA family
unique_family = []
for f in miRNA_family_dict:
    if not miRNA_family_dict[f] in unique_family:
        unique_family.append(miRNA_family_dict[f])

# create dataframe of gene and miRNA family to record the number of miRNA family for each gene
df_RNA_family = pd.DataFrame()
df_RNA_family['id'] = df_RNA['id']
for f in unique_family:
    df_RNA_family[f] = [str(i*0) for i in range(len(df_RNA['id']))]
    
miRNA = df_RNA['RNA_family']
transcript = df_RNA['id']
for mi,i in zip(miRNA,range(0,len(transcript))):
    miRNA_list = strTolist(mi)
    if len(miRNA_list)>0:
        for e in miRNA_list:
            this_family = miRNA_family_dict[e.split('.')[0]]
            pre_num = float(df_RNA_family.loc[i,this_family])
            df_RNA_family.loc[i,this_family] = pre_num + 1 
            
df_RNA_family.to_csv('C:/Users/yuyu/Desktop/Number_RNA_family.csv')
    
  

0: hsa-miR-29c-3p: mir-29
1: hsa-miR-29a-3p: mir-29
2: hsa-miR-29b-3p: mir-29
3: hsa-miR-9-5p: mir-9
4: hsa-miR-6504-5p: mir-6504
5: hsa-miR-3064-5p: mir-3064
6: hsa-miR-149-5p: mir-149
7: hsa-miR-199a-5p: mir-199
8: hsa-miR-199b-5p: mir-199
9: hsa-miR-195-5p: mir-15
10: hsa-miR-16-5p: mir-15
11: hsa-miR-15a-5p: mir-15
12: hsa-miR-15b-5p: mir-15
13: hsa-miR-424-5p: mir-322
14: hsa-miR-6838-5p: mir-6838
15: hsa-miR-497-5p: mir-497
16: hsa-miR-130a-5p: mir-130
17: hsa-miR-23c: mir-23c
18: hsa-miR-23a-3p: mir-23
19: hsa-miR-23b-3p: mir-23
20: hsa-miR-526b-3p: mir-515
21: hsa-miR-519d-3p: mir-515
22: hsa-miR-20a-5p: mir-17
23: hsa-miR-93-5p: mir-17
24: hsa-miR-17-5p: mir-17
25: hsa-miR-20b-5p: mir-17
26: hsa-miR-106a-5p: mir-17
27: hsa-miR-106b-5p: mir-17
28: hsa-miR-211-5p: mir-204
29: hsa-miR-204-5p: mir-204
30: hsa-miR-4319: mir-4319
31: hsa-miR-125b-5p: mir-10
32: hsa-miR-125a-5p: mir-10
33: hsa-miR-383-5p: mir-383
34: hsa-miR-543: mir-329
35: hsa-miR-504-5p: mir-504
36: hsa-miR-140-5p

291: hsa-miR-892c-3p: mir-743
292: hsa-miR-21-5p: mir-21
293: hsa-miR-590-5p: mir-590
294: hsa-miR-299-3p: mir-299
295: hsa-miR-216b-5p: mir-216
296: hsa-miR-6766-3p: mir-6766
297: hsa-miR-219a-5p: mir-219
298: hsa-miR-4782-3p: mir-4782
299: hsa-miR-216a-5p: mir-216
300: hsa-miR-190b: mir-190
301: hsa-miR-190a-5p: mir-190
302: hsa-miR-210-3p: mir-210
303: hsa-miR-147b: mir-147
304: hsa-miR-219a-2-3p: mir-219
305: hsa-miR-224-5p: mir-224
306: hsa-miR-668-3p: mir-668
307: hsa-miR-196a-5p: mir-196
308: hsa-miR-196b-5p: mir-196
309: hsa-miR-150-5p: mir-150
310: hsa-miR-215-5p: mir-192
311: hsa-miR-192-5p: mir-192
312: hsa-miR-370-5p: mir-370
313: hsa-miR-1193: mir-1193
314: hsa-miR-411-3p: mir-379
315: hsa-miR-379-3p: mir-379
316: hsa-miR-1251-5p: mir-1251
317: hsa-miR-758-3p: mir-379
318: hsa-miR-1224-5p: mir-1224
319: hsa-miR-532-5p: mir-188
320: hsa-miR-455-5p: mir-455
321: hsa-miR-1197: mir-379
322: hsa-miR-132-3p: mir-132
323: hsa-miR-212-3p: mir-132
324: hsa-miR-193a-3p: mir-193
325: